# select DRW dataset for SRNN

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl

In [62]:
names = locals()
names_id = locals()
for i in ['u','g','r','i','z']:
    df= pd.read_csv('s82drw/s82drw_'+i+'.csv')
    df['log_SF'] = np.array(df['log10(sigma[mag/sqrt(yr)])']).astype('float32')+0.5*(np.array(df['log10(tau[days])']).astype('float32') - np.log10(365))
    df= df.astype('float32')
    df['SF_'+i] = 10**df['log_SF']
    df = df.rename(columns={'log10(tau[days])':'log_tau', 'mu': 'mu_'+i})
    df['tau_'+i] = 10**df['log_tau']
    df = df[(df.edge_flag==0)&(df.Plike-df.Pnoise>2)&
               (df.Plike-df.Pinf> 0.05)&(df.log_tau<5)&
               (df.log_tau>0)&(df.SDR5ID != -1.0)]
    df = df.reset_index()
    df = df.drop(columns= ['index', 'Unnamed: 0', 'M_i', 'mass_BH[M/M_sun]', 'chi^2_pdf', 'log10(tau_lim_hi)', 
                      'log10(sig_lim_lo)', 'log10(sig_lim_hi)', 'log10(sigma[mag/sqrt(yr)])', 'log10(tau_lim_lo)',
                       'log_tau', 'log_SF', 'edge_flag', 'Plike', 'Pnoise', 'Pinf', 'npts'])
    names_id['df_'+i+'_id'] = df['SDR5ID']
    names['df_' + i] = df
    

In [63]:
# select rows in each df with same id, 3881 objects remained
common_id = list(set.intersection(*map(set, [df_u_id, df_g_id, df_r_id, df_i_id, df_z_id])))
for i in ['u','g','r','i','z']:
    names['df_' + i] = names['df_' + i][(names['df_' + i]['SDR5ID'].isin(common_id))]

In [64]:
df_u

,SDR5ID,ra,dec,redshift,mu_u,SF_u,tau_u
3,190.0,1.364696,-0.098956,0.6125,19.885900,0.364546,298.772003
5,49.0,0.281370,0.686984,1.3988,20.012800,0.244741,419.797607
7,229.0,1.669492,0.643952,1.3383,20.715700,0.302296,167.189926
8,699.0,4.850273,-0.276395,1.7690,20.158199,0.203779,380.285767
11,579.0,4.046200,-0.253447,1.5766,20.717199,0.329386,265.124573
...,...,...,...,...,...,...,...
6802,8442.0,56.149845,-0.257704,2.3452,20.216000,0.312272,1076.366089
6805,3820.0,25.001024,-1.200063,0.7830,19.217400,0.184757,139.350967
6807,6891.0,44.998692,0.803790,0.8923,19.948299,0.164354,109.015915
6808,6525.0,42.499062,-0.017835,1.5084,20.373100,0.185608,274.441711


In [79]:
# add estimated mu_y values
color_df = pd.read_csv('DR16/colour_redshift_corr.csv')
color_df['u-y'] = color_df['u-g']+color_df['g-r']+color_df['r-i']+color_df['i-z']+color_df['z-y']
color_df

,Unnamed: 0,low_z,high_z,u-g,g-r,r-i,i-z,z-y,u-y
0,0,0.00,0.05,1.343,0.028,1.082,0.263,-1.544,1.172
1,1,0.05,0.10,0.686,0.618,0.375,0.149,-0.585,1.243
2,2,0.10,0.15,0.881,0.571,0.407,0.218,-0.059,2.018
3,3,0.15,0.20,0.515,0.610,0.469,0.133,0.013,1.740
4,4,0.20,0.25,0.287,0.463,0.361,0.108,0.014,1.233
...,...,...,...,...,...,...,...,...,...
95,95,4.75,4.80,0.694,2.047,1.169,0.097,-0.146,3.861
96,96,4.80,4.85,-0.103,1.850,0.985,0.196,-0.324,2.604
97,97,4.85,4.90,0.506,1.768,1.027,0.156,-0.274,3.183
98,98,4.90,4.95,0.830,2.086,1.079,0.214,-0.223,3.986


In [96]:
# max(df_u['redshift'])

4.174699783325195

In [101]:
df_y = df_u.copy()
df_y['SF_u'] = df_y['SF_u']*0.61
df_y['tau_u'] = df_y['tau_u']*1.26
df_y = df_y.rename(columns={'SF_u':'SF_y', 'tau_u':'tau_y', 'mu_u':'mu_y'})
# print(df_y.iloc[1]['mu_y'])
for ind, row in df_y.iterrows():
    z = row['redshift']
    color_diff = color_df[(color_df.low_z<=z)&(color_df.high_z>z)]['u-y']
#     print(type(df_y.iloc[ind]['mu_y']))
    df_y['mu_y'][ind] =  df_y['mu_y'][ind] - color_diff
    



In [104]:
df_u

,SDR5ID,ra,dec,redshift,mu_u,SF_u,tau_u
3,190.0,1.364696,-0.098956,0.6125,19.885900,0.364546,298.772003
5,49.0,0.281370,0.686984,1.3988,20.012800,0.244741,419.797607
7,229.0,1.669492,0.643952,1.3383,20.715700,0.302296,167.189926
8,699.0,4.850273,-0.276395,1.7690,20.158199,0.203779,380.285767
11,579.0,4.046200,-0.253447,1.5766,20.717199,0.329386,265.124573
...,...,...,...,...,...,...,...
6802,8442.0,56.149845,-0.257704,2.3452,20.216000,0.312272,1076.366089
6805,3820.0,25.001024,-1.200063,0.7830,19.217400,0.184757,139.350967
6807,6891.0,44.998692,0.803790,0.8923,19.948299,0.164354,109.015915
6808,6525.0,42.499062,-0.017835,1.5084,20.373100,0.185608,274.441711


In [105]:
# combine 6 df together
full_df = df_u.merge(df_g, how='inner', on=['SDR5ID', 'ra', 'dec', 'redshift'])
full_df = df_r.merge(full_df, how='inner', on=['SDR5ID', 'ra', 'dec', 'redshift'])
full_df = df_i.merge(full_df, how='inner', on=['SDR5ID', 'ra', 'dec', 'redshift'])
full_df = df_z.merge(full_df, how='inner', on=['SDR5ID', 'ra', 'dec', 'redshift'])
full_df = df_y.merge(full_df, how='inner', on=['SDR5ID', 'ra', 'dec', 'redshift'])
full_df

,SDR5ID,ra,dec,redshift,mu_y,SF_y,tau_y,mu_z,SF_z,tau_z,...,tau_i,mu_r,SF_r,tau_r,mu_u,SF_u,tau_u,mu_g,SF_g,tau_g
0,190.0,1.364696,-0.098956,0.6125,18.591900,0.222373,376.452728,19.256599,0.252405,189.565811,...,680.957642,19.474199,0.336970,746.345764,19.885900,0.364546,298.772003,19.451401,0.350216,772.858337
1,49.0,0.281370,0.686984,1.3988,19.622801,0.149292,528.945007,19.625900,0.113709,208.646011,...,221.411438,19.719999,0.141630,338.922272,20.012800,0.244741,419.797607,19.909201,0.200131,414.009308
2,229.0,1.669492,0.643952,1.3383,20.289700,0.184401,210.659302,20.123699,0.167835,85.893433,...,228.738861,20.071600,0.150674,222.279800,20.715700,0.302296,167.189926,20.537100,0.263203,305.689148
3,699.0,4.850273,-0.276395,1.7690,19.726200,0.124305,479.160065,19.905600,0.247189,954.420837,...,1735.122192,20.231400,0.239701,795.005066,20.158199,0.203779,380.285767,20.109900,0.312402,1353.755005
4,579.0,4.046200,-0.253447,1.5766,20.176199,0.200925,334.056946,19.286699,0.212695,10.700399,...,7.141739,19.880400,0.184234,5.780734,20.717199,0.329386,265.124573,20.308001,0.207069,26.888090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3876,8442.0,56.149845,-0.257704,2.3452,19.270000,0.190486,1356.221313,19.017099,0.105224,191.156891,...,43.011044,19.568100,0.291977,2137.272949,20.216000,0.312272,1076.366089,19.510099,0.267904,1767.746826
3877,3820.0,25.001024,-1.200063,0.7830,18.354399,0.112702,175.582214,18.860500,0.119485,393.368866,...,429.190308,18.881201,0.163349,527.800781,19.217400,0.184757,139.350967,18.898100,0.160333,160.690460
3878,6891.0,44.998692,0.803790,0.8923,19.219299,0.100256,137.360046,19.209499,0.093306,252.964722,...,243.539810,19.359400,0.122705,325.769104,19.948299,0.164354,109.015915,19.538900,0.123066,137.733627
3879,6525.0,42.499062,-0.017835,1.5084,19.855101,0.113221,345.796539,19.744600,0.117581,865.465759,...,616.055603,20.014500,0.163477,793.067444,20.373100,0.185608,274.441711,20.156799,0.179214,563.728516


In [106]:
full_df.columns

Index(['SDR5ID', 'ra', 'dec', 'redshift', 'mu_y', 'SF_y', 'tau_y', 'mu_z',
       'SF_z', 'tau_z', 'mu_i', 'SF_i', 'tau_i', 'mu_r', 'SF_r', 'tau_r',
       'mu_u', 'SF_u', 'tau_u', 'mu_g', 'SF_g', 'tau_g'],
      dtype='object')

In [107]:
full_df.to_csv('SRNN_dataset/drw_3881.csv')

In [108]:
selected_df = full_df.sample(200)

In [109]:
selected_df.to_csv('SRNN_dataset/drw_200.csv')